# EdgeTTS

https://github.com/rany2/edge-tts

edge-tts is a Python module that allows you to use Microsoft Edge's online text-to-speech service from within your Python code or using the provided edge-tts or edge-playback command.

In [2]:
!pip install edge-tts
!pip install pygame

In [2]:
import asyncio
import edge_tts
import pygame

TEXT = """
cup, father, head, hat, above, mother, bird, sheep, ship, horse, blue, foot, day, eye, boy, nose, mouth, ear, hair, pure, fire, hour
"""

Wordlist = TEXT.split(",")

for w in Wordlist:
  for VOICE in ['en-US-GuyNeural', 'en-GB-RyanNeural']:
  # for VOICE in ['en-US-GuyNeural']:
    w = w.strip()
    OUTPUT_FILE = f"{w.replace(' ', '-').replace('?', '')}-{VOICE[3:5].replace('GB', 'UK').lower()}.mp3"
    communicate = edge_tts.Communicate(w, VOICE)
    await communicate.save(OUTPUT_FILE) 

print("Files created!") 

pygame 2.5.2 (SDL 2.28.3, Python 3.12.1)
Hello from the pygame community. https://www.pygame.org/contribute.html
Files created!


In [14]:
import asyncio
import edge_tts
import pygame

TEXT = "happy"
Wordlist = TEXT.split(",")

for w in Wordlist:
  for VOICE in ['zh-CN-YunxiNeural', 'zh-CN-XiaoxiaoNeural']:
    w = w.strip()
    OUTPUT_FILE = f"{w}-{VOICE.replace('YunxiNeural', 'male').replace('XiaoxiaoNeural', 'female').lower()}.mp3"
    communicate = edge_tts.Communicate(w, VOICE)
    await communicate.save(OUTPUT_FILE) 

print("Files created!") 

Files created!


In [ ]:
# VOICE = "en-US-GuyNeural" # Male
# VOICE = "en-US-AnaNeural" # Female
# VOICE = "en-US-AndrewNeural" # Male
# VOICE = "en-US-AriaNeural" # Female
# VOICE = "en-US-AvaNeural" # Female
# VOICE = "en-US-BrianNeural" # Male
# VOICE = "en-US-ChristopherNeural" # Male
# VOICE = "en-US-EmmaNeural" # Female
# VOICE = "en-US-EricNeural" # Male
# VOICE = "en-US-GuyNeural" # Male
# VOICE = "en-US-JennyNeural" # Female
# VOICE = "en-US-MichelleNeural" # Female
# VOICE = "en-US-RogerNeural" # Male
# VOICE = "en-US-SteffanNeural" # Male
# VOICE = "en-GB-LibbyNeural" # Female
# VOICE = "en-GB-MaisieNeural" # Female
# VOICE = "en-GB-RyanNeural" # Male
# VOICE = "en-GB-SoniaNeural" # Female
# VOICE = "en-GB-ThomasNeural" # Male
# VOICE = "en-AU-NatashaNeural" # Female
# VOICE = "en-AU-WilliamNeural" # Male
# VOICE = "en-CA-ClaraNeural" # Female
# VOICE = "en-CA-LiamNeural" # Male


In [ ]:
pygame.mixer.init()
pygame.mixer.music.load(OUTPUT_FILE)
pygame.mixer.music.play()

In [3]:
#American

import asyncio
import edge_tts
import pygame

TEXT = "hands"
Wordlist = TEXT.split(",")

for w in Wordlist:
  # for VOICE in ['en-US-GuyNeural', 'en-US-JennyNeural', 'en-GB-RyanNeural', 'en-GB-SoniaNeural']:
  for VOICE in ['en-US-GuyNeural', 'en-US-MichelleNeural']:
    w = w.strip()
    # OUTPUT_FILE = f"{w}-{VOICE.replace('EricNeural', 'Guy-Male').replace('JennyNeural', 'Jenny-Female').replace('RyanNeural', 'Ryan-Male').replace('SoniaNeural', 'Sonia-Female').lower()}.mp3"
    OUTPUT_FILE = f"{w}-{VOICE.replace('EricNeural', 'Male').replace('MichelleNeural', 'Female').replace('en-', '').lower()}.mp3"
    communicate = edge_tts.Communicate(w, VOICE)
    await communicate.save(OUTPUT_FILE) 

print("Files created!") 

Files created!


In [2]:
#American Sentence

import asyncio
import edge_tts
import pygame

TEXT = "cat,foot"
Wordlist = TEXT.split(",")

for w in Wordlist:
  # for VOICE in ['en-US-GuyNeural', 'en-US-JennyNeural', 'en-GB-RyanNeural', 'en-GB-SoniaNeural']:
  for VOICE in ['en-US-EricNeural', 'en-US-MichelleNeural']:
    w = w.strip().replace(' ', '-')
    # OUTPUT_FILE = f"{w}-{VOICE.replace('EricNeural', 'Guy-Male').replace('JennyNeural', 'Jenny-Female').replace('RyanNeural', 'Ryan-Male').replace('SoniaNeural', 'Sonia-Female').lower()}.mp3"
    OUTPUT_FILE = f"{w}-{VOICE.replace('EricNeural', 'Male').replace('MichelleNeural', 'Female').replace('en-', '').lower()}.mp3"
    communicate = edge_tts.Communicate(w, VOICE)
    await communicate.save(OUTPUT_FILE) 

print("Files created!") 

Files created!


In [ ]:
# ru-RU-DmitryNeural
# ko-KR-InJoonNeural

In [51]:
# British

import asyncio
import edge_tts
import pygame

TEXT = "speak, student, school, frustrate"
Wordlist = TEXT.split(",")

for w in Wordlist:
  # for VOICE in ['en-US-GuyNeural', 'en-US-JennyNeural', 'en-GB-RyanNeural', 'en-GB-SoniaNeural']:
  for VOICE in ['en-GB-RyanNeural', 'en-GB-SoniaNeural']:
    w = w.strip()
    # OUTPUT_FILE = f"{w}-{VOICE.replace('EricNeural', 'Guy-Male').replace('JennyNeural', 'Jenny-Female').replace('RyanNeural', 'Ryan-Male').replace('SoniaNeural', 'Sonia-Female').lower()}.mp3"
    OUTPUT_FILE = f"{w}-{VOICE.replace('RyanNeural', 'Male').replace('SoniaNeural', 'Female').replace('en-GB-', 'uk-').lower()}.mp3"
    communicate = edge_tts.Communicate(w, VOICE)
    await communicate.save(OUTPUT_FILE) 

print("Files created!") 

Files created!


In [49]:
#Korean

import asyncio
import edge_tts
import pygame

TEXT = "photo, video"
Wordlist = TEXT.split(",")

for w in Wordlist:
  # for VOICE in ['en-US-GuyNeural', 'en-US-JennyNeural', 'en-GB-RyanNeural', 'en-GB-SoniaNeural']:
  for VOICE in ['ko-KR-InJoonNeural']:
    w = w.strip()
    # OUTPUT_FILE = f"{w}-{VOICE.replace('EricNeural', 'Guy-Male').replace('JennyNeural', 'Jenny-Female').replace('RyanNeural', 'Ryan-Male').replace('SoniaNeural', 'Sonia-Female').lower()}.mp3"
    OUTPUT_FILE = f"{w}-{VOICE.replace('InJoonNeural', 'kr').replace('SoniaNeural', 'Female').replace('ko-KR', 'ko').lower()}.mp3"
    communicate = edge_tts.Communicate(w, VOICE)
    await communicate.save(OUTPUT_FILE) 

print("Files created!") 

Files created!


In [55]:
#Chinese

import asyncio
import edge_tts
import pygame

TEXT = "room"
Wordlist = TEXT.split(",")

for w in Wordlist:
  # for VOICE in ['en-US-GuyNeural', 'en-US-JennyNeural', 'en-GB-RyanNeural', 'en-GB-SoniaNeural']:
  for VOICE in ['zh-CN-liaoning-XiaobeiNeural']:
    w = w.strip()
    # OUTPUT_FILE = f"{w}-{VOICE.replace('EricNeural', 'Guy-Male').replace('JennyNeural', 'Jenny-Female').replace('RyanNeural', 'Ryan-Male').replace('SoniaNeural', 'Sonia-Female').lower()}.mp3"
    OUTPUT_FILE = f"{w}-{VOICE.replace('liaoning-XiaobeiNeural', 'zh').replace('SoniaNeural', 'Female').replace('zh-CN', 'cn').lower()}.mp3"
    communicate = edge_tts.Communicate(w, VOICE)
    await communicate.save(OUTPUT_FILE) 

print("Files created!") 

Files created!
